In [1]:
import faiss
import json
import numpy as np
import os
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import sys

# Add root directory (one level up from notebooks/)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

e:\School\Master's Program\Capstone\WikipediaNLP\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from utils.data_prep import traverse_directory, traverse_directory_to_single_json

input_directory = r'../data/processed/wikidata'
output_directory = r'../data/processed/wikidata_json'

traverse_directory(input_directory, output_directory)
traverse_directory_to_single_json(input_directory, output_directory + '.json')

In [ ]:
from utils.faiss_index import create_faiss_index, query_faiss

faiss_path = "../data/embeddings/faiss_index.index"
embedding_path = "../data/embeddings/article_embeddings.npy"

create_faiss_index(faiss_path, embedding_path)